In [33]:
#coding:utf-8
#data operation package 
import pandas as pd
import numpy as np


#show the result with picture
import seaborn as sns
from matplotlib import pyplot as plt
#show the datetime info
from datetime import datetime
from  sklearn  import  preprocessing

from sklearn.model_selection import train_test_split
from pre_data import convertDistance, convertRate
%matplotlib inline

In [34]:
train = pd.read_csv('datasets/offline_train.csv/processed_offline_train.csv')
select_columns=['User_id', 'Merchant_id', 'Coupon_id',  'Distance',
       'Date_received', 'Discount_rate']
X_data = train[select_columns]
X_data['Discount_rate'] = X_data['Discount_rate'].apply(convertRate)
X_data['Distance'] = X_data['Distance'].apply(convertDistance)
y_label = train['label']
print("X_data is ",np.shape(X_data))
print(X_data)
print(y_label.value_counts())

C:\Users\24582\AppData\Local\Temp\ipykernel_20576\1959929184.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_data['Discount_rate'] = X_data['Discount_rate'].apply(convertRate)


X_data is  (1053282, 6)
         User_id  Merchant_id  Coupon_id  Distance  Date_received   
0        1439408         4663    11002.0         1     20160528.0  \
1        1439408         2632     8591.0         0     20160217.0   
2        1439408         2632     1078.0         0     20160319.0   
3        1439408         2632     8591.0         0     20160613.0   
4        1439408         2632     8591.0         0     20160516.0   
...          ...          ...        ...       ...            ...   
1053277   212662         3021     3739.0         6     20160504.0   
1053278   212662         2934     5686.0         2     20160321.0   
1053279   212662         3021     3739.0         6     20160508.0   
1053280   752472         7113     1633.0         6     20160613.0   
1053281   752472         3621     2705.0         0     20160523.0   

         Discount_rate  
0                 0.87  
1                 0.95  
2                 0.95  
3                 0.95  
4                 0.95

C:\Users\24582\AppData\Local\Temp\ipykernel_20576\1959929184.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_data['Distance'] = X_data['Distance'].apply(convertDistance)


In [35]:
from sklearn.preprocessing import MinMaxScaler#导入包
min_max_scaler = MinMaxScaler()#创建模型
X_data_normal = pd.DataFrame(min_max_scaler.fit_transform(X_data),columns=X_data.columns)

In [36]:
from imblearn.over_sampling import SMOTE

In [37]:
smote = SMOTE(random_state=42)

In [38]:
print(X_data_normal.values)

[[0.19554388 0.52642873 0.78332384 0.18181818 0.8307393  0.84810127]
 [0.19554388 0.29704089 0.6116491  0.09090909 0.22568093 0.94936709]
 [0.19554388 0.29704089 0.07668755 0.09090909 0.42412451 0.94936709]
 ...
 [0.02888971 0.34097583 0.26616349 0.63636364 0.79182879 0.97468354]
 [0.10222322 0.80313982 0.11620621 0.63636364 0.99610895 0.75949367]
 [0.10222322 0.40874181 0.19253774 0.09090909 0.82101167 0.69620253]]


In [39]:
print(np.isnan(X_data_normal.values).any(axis=0))
# [ True  True False]

[False False False False False False]


In [40]:
print(~np.isnan(X_data_normal.values).any(axis=0))
# [False False  True]

[ True  True  True  True  True  True]


In [41]:
print(X_data_normal.values[:, ~np.isnan(X_data_normal.values).any(axis=0)])
# [[11. 14.]
#  [21. 24.]
#  [31. 34.]]

[[0.19554388 0.52642873 0.78332384 0.18181818 0.8307393  0.84810127]
 [0.19554388 0.29704089 0.6116491  0.09090909 0.22568093 0.94936709]
 [0.19554388 0.29704089 0.07668755 0.09090909 0.42412451 0.94936709]
 ...
 [0.02888971 0.34097583 0.26616349 0.63636364 0.79182879 0.97468354]
 [0.10222322 0.80313982 0.11620621 0.63636364 0.99610895 0.75949367]
 [0.10222322 0.40874181 0.19253774 0.09090909 0.82101167 0.69620253]]


In [42]:
X_train_res, y_train_res = smote.fit_resample(X_data_normal.values[:, ~np.isnan(X_data_normal.values).any(axis=0)], y_label.values)

In [43]:
def evaluate_mode(y_test,y_pre,mode,X_test):
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import auc
    acc= round(accuracy_score(y_test, y_pre)* 100, 2)
    pre= round(precision_score(y_test, y_pre)*100,2)
    rec= round(recall_score(y_test, y_pre)*100,2)
    f1=round(f1_score(y_test,y_pre)*100,2)
    from sklearn.metrics import roc_curve #导入ROC曲线函数
    fpr, tpr, thresholds = roc_curve(y_test, mode.predict_proba(X_test)[:,1], pos_label=1)
    roc_auc =round(auc(fpr,tpr),2)
    print("accourcy=%f\tpre=%f\trecall=%f\tf1=%f\tauc=%f\n"%(acc,pre,rec,f1,roc_auc))
    return acc,pre,rec,f1,roc_auc
#ROC曲线
def draw_ROC(X_test):
    from sklearn.metrics import roc_curve #导入ROC曲线函数
    fpr, tpr, thresholds = roc_curve(y_test, mode.predict_proba(X_test)[:,1], pos_label=1)
    plt.plot(fpr, tpr, linewidth=2, label = 'ROC of Liner', color = 'green') #作出ROC曲线
    plt.xlabel('False Positive Rate') #坐标轴标签
    plt.ylabel('True Positive Rate') #坐标轴标签
    plt.ylim(0,1.05) #边界范围
    plt.xlim(0,1.05) #边界范围
    plt.legend(loc=4) #图例

In [44]:
def linear(test):
    from sklearn.model_selection import train_test_split#留出法
    from sklearn.linear_model import LogisticRegression#线性回归分类器
    dt_rest=[]
    for times in range(5):
        X_train,X_test,y_train,y_test = train_test_split(X_train_res,y_train_res,test_size=0.2, random_state=42+times)
        #创建模型
        mode=LogisticRegression()
        mode.fit(X_train,y_train)
        y_pre=mode.predict(X_test)
        dt_rest.append(evaluate_mode(y_test,y_pre,mode,X_test))
    np.array(dt_rest).mean(axis=0)
    cls = mode.predict(test)
    prob = mode.predict_proba(test)
    return prob

In [45]:
def DT(test):
    from sklearn.tree import DecisionTreeClassifier
    mode=DecisionTreeClassifier(criterion="entropy",max_depth=10)
    #mode=DecisionTreeClassifier(criterion="gini",max_depth=10)
    dt_rest=[]
    for times in range(5):
        X_train,X_test,y_train,y_test = train_test_split(X_train_res,y_train_res,test_size=0.2, random_state=42+times)
        mode.fit(X_train,y_train)
        y_pre=mode.predict(X_test)
        dt_rest.append(evaluate_mode(y_test,y_pre,mode,X_test))
    np.array(dt_rest).mean(axis=0)
    cls = mode.predict(test)
    prob = mode.predict_proba(test)
    return prob

In [46]:
def RF(test):
    from sklearn.ensemble import RandomForestClassifier
    dt_rest=[]
    for n_estimator in range(10,20,10):
        rf_rest=[]
        mode = RandomForestClassifier(n_estimators=n_estimator,criterion="entropy",max_depth=10,max_features=3)
        for times in range(5):
            X_train,X_test,y_train,y_test = train_test_split(X_train_res,y_train_res,test_size=0.2, random_state=42+times)
        mode.fit(X_train,y_train)
        y_pre=mode.predict(X_test)
        dt_rest.append(evaluate_mode(y_test,y_pre,mode,X_test))
        
    np.array(dt_rest).mean(axis=0)
    cls = mode.predict(test)
    prob = mode.predict_proba(test)
    return prob

In [47]:
test = pd.read_csv('datasets/offline_test.csv/offline_test.csv')
from pre_data import convertDistance, convertRate
test['Discount_rate'] = test['Discount_rate'].apply(convertRate)
test['Distance'] = test['Distance'].apply(convertDistance)
# print(test)
test_normal = pd.DataFrame(min_max_scaler.fit_transform(test),columns=test.columns)
print(test_normal.values)
print(~np.isnan(test_normal.values).any(axis=0))

[[0.56098788 0.05016949 0.71072497 0.75757576 0.18181818 0.36666667]
 [0.94407603 0.14621469 0.24398234 0.75757576 0.         0.16666667]
 [0.29430437 0.80305085 0.49316337 0.86363636 0.54545455 0.86666667]
 ...
 [0.90025412 0.82655367 0.54030765 0.86363636 0.         0.36666667]
 [0.61771149 0.08519774 0.96845179 0.75757576 0.09090909 0.53333333]
 [0.90693164 0.84531073 0.04344111 0.93939394 0.54545455 0.9       ]]
[ True  True  True  True  True  True]


In [48]:
from sklearn.tree import DecisionTreeClassifier
mode=DecisionTreeClassifier(criterion="entropy",max_depth=10)
    #mode=DecisionTreeClassifier(criterion="gini",max_depth=10)
dt_rest=[]
pre_res_prob = []
pre_res = []
for times in range(5):
    X_train,X_test,y_train,y_test = train_test_split(X_train_res,y_train_res,test_size=0.2, random_state=42+times)
    mode.fit(X_train,y_train)
    # print(X_train)
    y_pre=mode.predict(X_test)
    # print(X_test)
    dt_rest.append(evaluate_mode(y_test,y_pre,mode,X_test))
    test_predict_prob = mode.predict_proba(test_normal.values)
    test_predict = mode.predict(test_normal.values)
    pre_res_prob.append(test_predict_prob)
    pre_res.append(test_predict)
# mode.predict()
# mode.predict_proba()


accourcy=77.200000	pre=73.320000	recall=85.660000	f1=79.010000	auc=0.850000

accourcy=77.080000	pre=73.240000	recall=85.260000	f1=78.790000	auc=0.850000

accourcy=77.170000	pre=73.750000	recall=84.440000	f1=78.730000	auc=0.850000

accourcy=77.250000	pre=73.400000	recall=85.640000	f1=79.050000	auc=0.850000

accourcy=77.110000	pre=72.990000	recall=85.820000	f1=78.890000	auc=0.850000



In [49]:
print(len(pre_res[0]),len(pre_res[1]))
print(pre_res_prob)
print(pre_res)

113640 113640
[array([[0.27419355, 0.72580645],
       [0.66536661, 0.33463339],
       [0.44383219, 0.55616781],
       ...,
       [0.83333333, 0.16666667],
       [0.27419355, 0.72580645],
       [0.18329596, 0.81670404]]), array([[0.51818182, 0.48181818],
       [0.66769231, 0.33230769],
       [0.43866499, 0.56133501],
       ...,
       [0.66666667, 0.33333333],
       [0.15966387, 0.84033613],
       [0.17765363, 0.82234637]]), array([[0.68421053, 0.31578947],
       [0.7483871 , 0.2516129 ],
       [0.43985389, 0.56014611],
       ...,
       [1.        , 0.        ],
       [0.21495327, 0.78504673],
       [0.18062397, 0.81937603]]), array([[0.23404255, 0.76595745],
       [0.66564652, 0.33435348],
       [0.44280675, 0.55719325],
       ...,
       [0.83333333, 0.16666667],
       [0.23404255, 0.76595745],
       [0.18063818, 0.81936182]]), array([[0.29381443, 0.70618557],
       [0.66036249, 0.33963751],
       [0.44225423, 0.55774577],
       ...,
       [0.83333333, 0.1666

In [50]:
from sklearn.ensemble import RandomForestClassifier
dt_rest=[]
for n_estimator in range(10,20,10):
    rf_rest=[]
    mode = RandomForestClassifier(n_estimators=n_estimator,criterion="entropy",max_depth=10,max_features=3)
    for times in range(5):
        X_train,X_test,y_train,y_test = train_test_split(X_train_res,y_train_res,test_size=0.2, random_state=42+times)
    mode.fit(X_train,y_train)
    y_pre=mode.predict(X_test)
    dt_rest.append(evaluate_mode(y_test,y_pre,mode,X_test))
    test_predict_prob = mode.predict_proba(test)

accourcy=78.850000	pre=74.310000	recall=87.970000	f1=80.570000	auc=0.880000



e:\Anaconda3\envs\d2l-zh\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [72]:

import tkinter as tk
from tkinter import ttk
from pre_data import convertDistance, convertRate


class APP:
    def __init__(self,selected_option=None):
        self.selected_option = selected_option
        #if self.selected_opti == Option 1 
           # linear()
        #print(self.selected_option)

    def on_button_click(self, info):
        print("Yes button clicked!",self.selected_option)
        # infomation = [
        #                 float(user_id), float(merchant_id), float(coupon_id), 
        #                 convertRate(discount_rate), convertDistance(distance), float(date)
        #               ]
        # info = pd.DataFrame(infomation)
        if self.selected_option == "linear":
            prob3 = linear(info)
        if self.selected_option == "DT":
            prob3 = DT(info)
        if self.selected_option == "RF":
            prob3 = RF(info)
        return prob3

class GUI:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("User Info")
        self.root.geometry("600x450")
        self.create_widgets()

    def on_yes_button_click(self, user_id, merchant_id, coupon_id, distance, discount_rate, date):
        selected_option = self.parameter.get()
        print("call the app with the option selected!", selected_option)
        # print(info)
        if distance == 'null':
            infomation = {
                            'User_id' : float(user_id),
                            'Merchant_id' : float(merchant_id), 
                            'Coupon_id' : float(coupon_id),  
                            'Distance' : None,
                            'Date_received' : convertRate(discount_rate), 
                            'Discount_rate' : float(date),  
                         }          
        else:
            infomation = {
                            'User_id' : float(user_id),
                            'Merchant_id' : float(merchant_id), 
                            'Coupon_id' : float(coupon_id),  
                            'Distance' : convertDistance(distance),
                            'Date_received' : convertRate(discount_rate), 
                            'Discount_rate' : float(date),  
                         }       
        info = pd.DataFrame(infomation, index=[0])
        # info['User_id'] = float(user_id)
        # info['Merchant_id'] = float(merchant_id)
        # info['Coupon_id'] = float(coupon_id)
        # info['Discount_rate'] = convertRate(discount_rate)
        # if distance == 'null':
        #     info['Distance'] = np.nan
        # else:
        #     info['Distance'] = convertDistance(distance)
        # info['Date_received'] = float(date)
        print(info)
        # info_normal = pd.DataFrame(min_max_scaler.fit_transform(info),columns=info.columns)
        # print(info_normal)

        app = APP(selected_option)
        res = app.on_button_click(info.values)
        self.show_res(res)

    def show_res(self, res):
        # self.parameter = tk.StringVar()
        var_res_prob = tk.StringVar()
        
        print(res)
        print(res[0][1])
        date_label = ttk.Label(self.root, text="Prob:")
        date_label.place(x=50,y=210)
        var_res_prob.set(str(res[0][1]))

        res_label = tk.Label(self.root, textvariable=var_res_prob)
        res_label.pack()
        # var_res_prob.set(str(res[0][1]))
        res_label.place(x=130, y=210)

    def close_window(self):
        self.root.destroy()

    def create_widgets(self):
        self.parameter = tk.StringVar()

        # User ID Label
        var_user_id = tk.StringVar()
        user_id_label = ttk.Label(self.root, text="User ID:")
        user_id_label.place(x=50,y=50)

        entry_user_id = ttk.Entry(self.root, textvariable=var_user_id)
        entry_user_id.place(x=130, y=50)
        

        var_merchant_id = tk.StringVar()
        merchant_id_label = ttk.Label(self.root, text="Merchant ID:")
        merchant_id_label.place(x=300,y=50)

        entry_merchant_id = ttk.Entry(self.root, textvariable=var_merchant_id)
        entry_merchant_id.place(x=380, y=50)

        var_discount_rate = tk.StringVar()
        discount_rate = ttk.Label(self.root, text="Discount rate:")
        discount_rate.place(x=300,y=80)

        entry_discount_rate = ttk.Entry(self.root, textvariable=var_discount_rate)
        entry_discount_rate.place(x=390, y=80)

        var_distance = tk.StringVar()
        distance = ttk.Label(self.root, text="Distance:")
        distance.place(x=300,y=110)

        entry_distance = ttk.Entry(self.root, textvariable=var_distance)
        entry_distance.place(x=380, y=110)

        var_coupon_id = tk.StringVar()
        coupon_id_label = ttk.Label(self.root, text="Coupon ID:")
        coupon_id_label.place(x=50,y=80)

        entry_coupon_id = ttk.Entry(self.root, textvariable=var_coupon_id)
        entry_coupon_id.place(x=130, y=80)

        var_date = tk.StringVar()
        date_label = ttk.Label(self.root, text="Date received:")
        date_label.place(x=50,y=110)

        entry_date = ttk.Entry(self.root, textvariable=var_date)
        entry_date.place(x=130, y=110)

        # Combobox with some options
        combobox_label = ttk.Label(self.root, text="Combobox:")
        combobox_label.place(x=50,y=140)

        options = ["linear", "DT", "RF"]
        combo_box = ttk.Combobox(self.root, values=options, textvariable=self.parameter)
        combo_box.place(x=130, y=140)

        # Yes and No buttons to submit all input information
        # var = entry_user_id.get() == '' or entry_merchant_id.get() == '' or entry_coupon_id.get() == '' or entry_distance.get() == '' or entry_discount_rate.get() == '' or entry_date.get() == ''
        # while var:
        #     pass
        # infomation = [
        #                 float(entry_user_id.get()), float(entry_merchant_id.get()), float(entry_coupon_id.get()), 
        #                 convertRate(entry_discount_rate.get()), convertDistance(entry_distance.get()), float(entry_date.get())
        #               ]
        # info = pd.DataFrame(infomation)
        yes_button = ttk.Button(self.root, text="Submit", command=lambda:self.on_yes_button_click(entry_user_id.get(), entry_merchant_id.get(),
                                                                                                  entry_coupon_id.get(), entry_distance.get(),
                                                                                                  entry_discount_rate.get(), entry_date.get()))
        yes_button.place(x=50, y=180)


        no_button = ttk.Button(self.root, text="Cancel", command=self.close_window)
        no_button.place(x=150,y=180)


if __name__ == "__main__":
    gui = GUI()
    gui.root.mainloop()

call the app with the option selected! RF
     User_id  Merchant_id  Coupon_id  Distance  Date_received  Discount_rate
0  4129537.0        450.0     9983.0         1           0.83     20160712.0
Yes button clicked! RF
accourcy=79.130000	pre=74.620000	recall=88.070000	f1=80.790000	auc=0.880000

[[0.48908316 0.51091684]]
0.510916838967706
